# OpenIntro Stats Exercises

The following is a series of exercises, taken from OpenIntro Stats 4ed, for the purposes of developing my own understanding of the statistical tests alongside increasing familiarity with common statistical packages.

In [14]:
import scipy.stats
import numpy as np
import pandas as pd

## Chpt 6

### 6.2 Hypothesis Tests for the Difference of Two Sample Means

#### 6.13 Guided Practice

In [164]:
# Create the dataframe given in the question

df = pd.DataFrame({'heart_attack' : [145, 200], 'no_event' : [12788, 12738]} , index = ['fish_oil', 'placebo'])
df['total'] = df['heart_attack'] + df['no_event']
df


,heart_attack,no_event,total
fish_oil,145,12788,12933
placebo,200,12738,12938


First verify that a normal approximation is valid. Each subject was randomly assigned to one of the two groups so we may assume independence holds. Now we verify the success-failure condition:

In [166]:
# Fish oil group:

p1_hat , p2_hat = df['heart_attack'] / df['total'] 
n1, n2 = df['total']

all([n1*p1_hat>10, n1*(1-p1_hat)>10, n2*p2_hat>10, n2*(1-p2_hat)])

True

All good here. We are safe to use a normal distribution to approximate the sampling distribution. We now construct the 95% confidence interval.

In [174]:
point_estimate = p1_hat - p2_hat
std_error = np.sqrt( p1_hat*(1-p1_hat)/n1 + p2_hat*(1-p2_hat)/n2 )

# Compute corresponding z-value using an inverse CDF of a standard normal distribution:

z = stats.norm.ppf(0.975)

# Convert to float dtype for more aesthetic printing

print([float(point_estimate-z*std_error), float(point_estimate+z*std_error)])

[-0.007041644824162001, -0.0014517763930541822]


Observing the confidence interval, we have evidence to reject the hypothesis that the fish oil treatment does not affect heart attack outcomes.

#### 6.15 Guided Practice

Not exactly what Ex 6.15 asks for but a good exercise nonetheless.

In [185]:
# Create the data given:

df = pd.DataFrame({'Yes':[500,505], 'No':[44425,44405]} )
df.index = ['Mammogram','Control']
df

,Yes,No
Mammogram,500,44425
Control,505,44405


Let $p_1\, \, p_2$ be the true proportion of those who die of cancer in the mammogram and control groups respectively. We wish to test if there is a statistically significant difference between the two proportions. We state our null and alternate hypotheses as:

$
H_0 : p_1 - p_2 = 0 
$

$
H_1 : p_1 - p_2 \neq 0
$

We will conduct a two-tailed test at the 5% significance level. We first verify that a normal distribution is a valid approxiation of the sample distribution in this case:

In [199]:
n1, n2 = df['Yes'] + df['No']
p1, p2 = df['Yes'] / [n1, n2]

all([n1*p1>10, n1*(1-p1)>10, n2*p2>10, n2*(1-p2)>10])

True

The sample parameters seem reasonable to use a normal distribution as an approximation. Since the women were randomly assigned to one of the two groups, we may assume independence holds both within each group and across each group.

We now compute the p-value for our observed statistic:

In [225]:
mu = 0

# Estimate the standard deviation of the null distribution using the sample parameters
sigma = np.sqrt( p1*(1-p1)/n1 + p2*(1-p2)/n2 )

# p1-p2<0 so we compute the p-value using symmetries of the normal distribution

p = stats.norm(mu, sigma).cdf(p1-p2)
float(p)

0.4348920065167426

This is clearly above the 2.5% mark (two-tailed test at 95% significance) so we have no evidence to reject the null hypothesis.